### LLL rácsredukció
A.K. Lenstra, H.W. Lenstra és Lovász L. mutattak be egy hatékony és híres redukciós kritériumot tetszőleges rácsméretekre, és az általuk javasolt algoritmus LLL algoritmus néven ismert. Ez a Gauss-redukció kiterjesztéseként értelmezhető m > 2 rangú rácsokra.

Az alábbiakban egy egyszerű LLL implementációt mutatunk be Python nyelven.

In [1]:
import numpy as np

def LLL(A, delta=0.75):
    # Input: A     : input bázis
    #        delta : LLL redukció paramétere
    # Output:    B : LLL-redukált bázis, B = A*T
    #            T : unimoduláris transzformáló mátrix
    if delta > 1 or delta < 0.25:
        print("A delta értéke érvénytelen, helyette 0.75-öt használunk.")
        delta = 0.75
    # initialization
    B     = np.copy(A)                 # redukált B mátrix
    Q,R   = np.linalg.qr(B,'reduced')  # B mátrix QR felbontása
    (n,m) = B.shape                    # mátrix alaka
    T     = np.eye(m)                  # unimoduláris m x m -es mátrix
    
    # LLL - redukció
    l  = 1
    while l < m:
        # A B[:,l] oszlopvektor méretének csökkentése
        for k in range(l-1,-1,-1):
            mu = np.round(R[k,l]/R[k,k])       # abs(R[k,l])>0.5*abs(R[k,k])
            if np.abs(mu)>0:
                B[:,l]     = B[:,l]   - mu * B[:,k]
                R[:k+1,l]  = R[:k+1,l]  - mu * R[:k+1,k]
                T[:,l]     = T[:,l]   - mu * T[:,k]
        # Lovász feltétel
        lng = np.linalg.norm(R[l-1:l+1,l])
        if delta*np.abs(R[l-1,l-1])**2 > lng**2:
            # az l-1 és l oszlopok felcserélése B-ben, T-ben és R-ben
            B[:,[l-1,l]]     = B[:,[l,l-1]]
            T[:,[l-1,l]]     = T[:,[l,l-1]]
            R[:l+1,[l-1,l]]  = R[:l+1,[l,l-1]]
            # a felső háromszögszerkezet felépítése Givens-forgatással 
            # a Theta mátrixszal történő szorzás eredményeképpen R[l,l-1] = 0
            c     = R[l-1,l-1] / lng        # lng = ||R[l-1:l,l-1]|| felcserélés után
            s     = R[l,l-1]   / lng
            Theta = np.array([[c, s], [-s, c]])
            R[l-1:l+1,l-1:] = np.dot(Theta, R[l-1:l+1,l-1:])
            Q[:,l-1:l+1]      = np.dot(Q[:,l-1:l+1], Theta.T)
            l                 = max(l-1,1)
        else:
            l = l+1

    return B,T

Ellenőrizzük a kódunkat egy egyszerű példával [innen](https://en.wikipedia.org/wiki/Lenstra%E2%80%93Lenstra%E2%80%93Lov%C3%A1sz_lattice_basis_reduction_algorithm).

In [2]:
# tesztpélda a Wikipédiából
A = np.array([[1,-1,3],[1,0,5],[1,2,6]])
print("kezdeti rácsbázis:")
print(A)
# make LLL reduction
B,T = LLL(A)
print("LLL redukált bázis:")
print(B)
print("unimoduláris transzformáló mátrix:")
print(T)

kezdeti rácsbázis:
[[ 1 -1  3]
 [ 1  0  5]
 [ 1  2  6]]
LLL redukált bázis:
[[ 0  1 -1]
 [ 1  0  0]
 [ 0  1  2]]
unimoduláris transzformáló mátrix:
[[-4.  5.  0.]
 [-1.  1.  1.]
 [ 1. -1.  0.]]


Most kiszámítjuk a Babai-pontot Babai kerekítési technikájával.

In [3]:
# input point
t = np.array([4.2,7.1,-2.6])
print("input pont t: ")
print(t)
# Babai's rounding technique
l = np.dot(t,np.linalg.inv(B))
v = np.dot(np.round(l),B)
print("Babai pont v: ")
print(v)
tv = t-v
print("A t-v euklideszi normája: ")
print("{:.4f}".format(np.linalg.norm(t-v)))
m = np.dot(tv,np.linalg.inv(B))
print("a t-v koordinátái LLL-bázisban: ")
print(m)

input pont t: 
[ 4.2  7.1 -2.6]
Babai pont v: 
[ 4.  7. -4.]
A t-v euklideszi normája: 
1.4177
a t-v koordinátái LLL-bázisban: 
[-0.4  0.2  0.5]
